<a href="https://colab.research.google.com/github/descandon88/tesis_maestria/blob/main/analisis_mozambique/comparativos_paquetes_ecomplexity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd
import numpy as np
# Librerías para visualizacion
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
import re
import networkx as nx
from itertools import count
from itertools import combinations
from itertools import product
import statsmodels.api as sm
import urllib.request, json
!pip3 install geopandas
import geopandas
# -- set scientific notation to display numbers fully rather than exponential
pd.set_option('display.float_format', '{:.2f}'.format)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' 
import seaborn as sns
sns.set_style('whitegrid') # grids en dark background

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 100000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth',300)
print('librerías cargadas')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.4 MB/s eta 0:00:00
librerías cargadas


# Análisis de los datos contrade HS4 



Se importa la base desde drive

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [57]:
## Se importa la base de exportaciones de productos drive
df_base = pd.read_csv("/content/drive/MyDrive/eci/base/df_updated.csv")
df_short_name = pd.read_csv("/content/drive/MyDrive/eci/base/df_tipo_pro.csv")

print('dataframes cargados')

dataframes cargados


In [5]:
df_base.describe()

,year,product_code,export_value
count,7373271.00,7373271.00,7373271.00
mean,2007.65,5304.58,42867657.55
std,7.44,2767.02,899054286.46
min,1995.00,101.00,0.00
25%,2001.00,2843.00,0.00
50%,2008.00,5402.00,1119.00
75%,2014.00,8102.00,432040.50
max,2020.00,9999.00,371679199232.00


In [54]:
# df_base['product_code'] = df_base['product_code'].apply(lambda x: '{0:0>4}'.format(x))
# df['product_code']  = df['product_code'] .apply(np.int64)

df_base.sample(n=6)
df_base.info()

,year,country_code,country_name,product_code,product_name,export_value
910457,2011,SEN,Senegal,1704,"Sugar confectionery (including white chocolate), not containing cocoa",1226236
2036354,2000,GTM,Guatemala,2925,Carboxyimide-function compounds (including saccharin and its salts) and imine-function compounds,8189
6187656,2010,CMR,Cameroon,8474,"Machinery for sorting, screening, separating, washing, crushing, grinding, mixing or kneading earth, stone, ores in solid form, shaping, moulding machinery for solid mineral fuels",1668605
5446162,2006,AIA,Anguilla,7902,Zinc; waste and scrap,0
3518913,1998,KIR,Kiribati,5110,"Yarn of coarse animal hair or of horsehair (including gimped horsehair yarn), whether or not put up for retail sale",0
2265505,2007,TCA,Turks and Caicos Islands,3210,"Paints and varnishes (including enamels, lacquers and distempers), excluding those of heading no. 3209, prepared water pigments of a kind used for finishing leather",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7373271 entries, 0 to 7373270
Data columns (total 6 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   year          int64 
 1   country_code  object
 2   country_name  object
 3   product_code  int64 
 4   product_name  object
 5   export_value  int64 
dtypes: int64(3), object(3)
memory usage: 337.5+ MB


In [55]:
# df_test.sample(n=6)
# df_test['hs_product_code'] = df_test['hs_product_code'].apply(np.int64)
df_short_name.rename(columns={"hs_product_code": "product_code"}, inplace=True)
df_short_name.rename(columns={"hs_product_name_short_en": "product_short_name"}, inplace=True)


df_short_name.sample(n=15)

df_short_name.info()
df_short_name['product_id'].nunique()
df_short_name['product_short_name'].nunique()

df_short_name.describe()

,product_id,product_code,product_short_name
41,130,31,Fertilisers
3569,7208,520641,"Cotton yarn <85%, mult comb >714"
1288,1825,9103,Clocks with watch movements
3141,6780,400819,"Vulcanised rubber rods, cellular"
2678,6317,293500,Sulphonamides
3839,7478,551522,"Woven fabric acrylics & wool/hair, nes"
5541,9180,847193,Computer data storage units
1929,5568,190110,"Infant foods of cereals, milk"
3645,7284,521213,"Cotton fabric, <200g/m2, dye, nes"
4129,7768,620312,"Mens suits, synthetic fibres, not knit"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6406 entries, 0 to 6405
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   product_id          6406 non-null   int64 
 1   product_code        6406 non-null   object
 2   product_short_name  6406 non-null   object
dtypes: int64(1), object(2)
memory usage: 150.3+ KB


6406

6252

,product_id
count,6406.00
mean,6176.96
std,2905.80
min,0.00
25%,5240.25
50%,6841.50
75%,8442.75
max,11004.00


### Aplico un merge para agregar el nombre corto del producto al dataframe principal

In [56]:
# df_base['product_code'].nunique()
df_base2018 = df_base[df_base['year']==2018].copy()
print("# de productos 2018 en la df_base: ",df_base2018['product_name'].nunique())
print("# de paises 2018 en la df_base: ",df_base2018['country_name'].nunique())

    # df_all = pd.merge(df_all,data[[anio_col,pais_col,prod_col,prod_code,value_col]],how='left',on=[anio_col,pais_col,prod_col])

# df['product_code'].nunique()
df = pd.merge(df_base, df_short_name[['product_code','product_short_name']],how='inner', on=['product_code']).copy()
# df = pd.merge(df_base2018, df_short_name[['product_code','product_short_name']],how='inner', on='product_code').copy()
print("# de productos 2018 en la df: ",df['product_name'].nunique())
print("# de paises 2018 en la df: ",df['country_name'].nunique())
df.head()

# de productos 2018 en la df_base:  1220
# de paises 2018 en la df_base:  235


ValueError: ignored

In [9]:
df_prueba = df[(df['product_code']=='0306')].copy()
df_prueba.head()

,year,country_code,country_name,product_code,product_name,export_value,product_short_name
125202,1995,ABW,Aruba,0306,"Crustaceans, in shell or not, live, fresh, chilled, frozen, dried, salted or in brine; crustaceans, in shell, cooked by steaming or boiling in water, chilled or not, frozen, dried, salted or in brine",0,Crustaceans
125203,1996,ABW,Aruba,0306,"Crustaceans, in shell or not, live, fresh, chilled, frozen, dried, salted or in brine; crustaceans, in shell, cooked by steaming or boiling in water, chilled or not, frozen, dried, salted or in brine",1408,Crustaceans
125204,1997,ABW,Aruba,0306,"Crustaceans, in shell or not, live, fresh, chilled, frozen, dried, salted or in brine; crustaceans, in shell, cooked by steaming or boiling in water, chilled or not, frozen, dried, salted or in brine",6620,Crustaceans
125205,1998,ABW,Aruba,0306,"Crustaceans, in shell or not, live, fresh, chilled, frozen, dried, salted or in brine; crustaceans, in shell, cooked by steaming or boiling in water, chilled or not, frozen, dried, salted or in brine",56542,Crustaceans
125206,1999,ABW,Aruba,0306,"Crustaceans, in shell or not, live, fresh, chilled, frozen, dried, salted or in brine; crustaceans, in shell, cooked by steaming or boiling in water, chilled or not, frozen, dried, salted or in brine",242243,Crustaceans


Finding specific countries / products based on partial string matching
If you’re interested in finding data on certain countries / products but not sure how exactly these are spelled in the data (or are spelled with / without e.g. capital letters)

In [10]:
STRING = 'Netherland'
df_base[df_base['country_name'].str.contains(STRING)][['country_name']].drop_duplicates()

STRING = 'wine'
df_base[df_base['product_name'].str.contains(STRING,flags=re.IGNORECASE, regex=True)][['product_name']].drop_duplicates()

,country_name
156,Netherlands Antilles
3711,Netherlands


,product_name
11924,Swine; live
47696,"Meat of swine; fresh, chilled or frozen"
65582,"Edible offal of bovine animals, swine, sheep, goats, horses, asses, mules or hinnies; fresh, chilled or frozen"
1084954,"Wine of fresh grapes, including fortified wines; grape must other than that of heading no. 2009"
1090916,"Vermouth and other wine of fresh grapes, flavoured with plants or aromatic substances"
1156498,Wine lees; argol
3855058,"Twine, cordage, ropes and cables, whether or not plaited or braided; whether or not impregnated, coated, covered or sheathed with rubber or plastics"
3861020,"Twine, cordage or rope; knotted netting, made up fishing nets and other made up nets, of textile materials"
3866982,"Articles of yarn, strip or the like of heading no. 5404 or 5405; twine, cordage, rope or cables n.e.s. or included"
4302208,"Rags; used or new, scrap twine, cordage, rope and cables and worn out articles of twine, cordage, rope or cables, of textile materials"


# Práctica de los paquetes y librerías

# Cálculo de Complejidad Económica

In [11]:
!pip install ecomplexity
print('py-ecomplexity instalado')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ecomplexity: filename=ecomplexity-0.5.2-py3-none-any.whl size=11283 sha256=ea10a524226968185f64d02bc8d756a20c5bba72ca81cd69dc511bfc386aea85
  Stored in directory: /root/.cache/pip/wheels/c7/14/29/d28c7286333b75599f8f60681634819c63330cc3b89cf85f61
Successfully built ecomplexity
py-ecomplexity instalado


In [58]:
from ecomplexity import ecomplexity
from ecomplexity import proximity

# To use py-ecomplexity, specify the following columns
# cols_input: dict of column names for time, location, product and value.
    #Example: {'time':'year', 'loc':'origin', 'prod':'hs92', 'val':'export_val'}
    
trade_cols = {'time':'year',
              'loc':'country_name',
              'prod':'product_name',
              'val':'export_value'}

print('Calculando complejidad económica')
df_ec = ecomplexity(df_base, trade_cols)
print('Cálculo finalizado')



Calculando complejidad económica
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
Cálculo finalizado


In [59]:
df_complexity = df_ec[['country_name',
               'product_name',
               'product_code',
               'export_value',
               'rca',
               'mcp',
               'density',
               'year',
               'pci',
               'eci',
               'coi',
               'cog']].copy()



In [60]:
# Muestro resultados
df_complexity.head(n=10)

,country_name,product_name,product_code,export_value,rca,mcp,density,year,pci,eci,coi,cog
0,Afghanistan,"Abrasive powder or grain; natural or artificial, on a base of textile material, of paper, paperboard or of other material, whether or not cut to shape or sewn or otherwise made up",6805,1327,0.05,0,0.04,1995,3.01,-0.33,-0.59,1.37
1,Afghanistan,Accordions and similar instruments; mouth organs,9204,0,0.00,0,0.03,1995,3.88,-0.33,-0.59,1.05
2,Afghanistan,"Acetals and hemiacetals; whether or not with other oxygen function, and their halogenated, sulphonated, nitrated or nitrostated derivatives",2911,0,0.00,0,0.03,1995,3.77,-0.33,-0.59,1.42
3,Afghanistan,"Acids; carboxylic acid with additional oxygen function and their anhydrides, halides, peroxides, peroxyacids; their halogenated, sulphonated, nitrated or nitrosated derivatives",2918,0,0.00,0,0.04,1995,2.37,-0.33,-0.59,0.99
4,Afghanistan,"Acids; polycarboxylic acids, their anhydrides, halides, peroxides and peroxy-acids; their halogenated, sulphonated, nitrated or nitrosated derivatives",2917,0,0.00,0,0.05,1995,1.50,-0.33,-0.59,0.74
5,Afghanistan,"Acids; saturated acyclic monocarboxylic acids and their anhydrides, halides, peroxides and peroxyacids; their halogenated, sulphonated, nitrated or nitrosated derivatives",2915,0,0.00,0,0.03,1995,3.03,-0.33,-0.59,1.30
6,Afghanistan,"Acids; unsaturated acyclic monocarboxylic, cyclic monocarboxylic , their anhydrides, halides, peroxides and peroxyacids; their halogenated, sulphonated, nitrated or nitrosated derivatives",2916,0,0.00,0,0.03,1995,3.33,-0.33,-0.59,1.51
7,Afghanistan,Acrylic polymers in primary forms,3906,0,0.00,0,0.03,1995,3.76,-0.33,-0.59,1.19
8,Afghanistan,"Activated carbon; activated natural mineral products; animal black, including spent animal black",3802,0,0.00,0,0.05,1995,1.13,-0.33,-0.59,0.65
9,Afghanistan,"Acyclic alcohols and their halogenated, sulphonated, nitrated or nitrosated derivatives",2905,0,0.00,0,0.05,1995,1.01,-0.33,-0.59,0.46


In [61]:
df_2018 = df_complexity[df_complexity['year']==2018].copy()
moz2018 = df_2018[df_2018['country_name']=='Mozambique'].copy()
moz2018=moz2018.sort_values(by=['product_code'])

moz2018.head()

,country_name,product_name,product_code,export_value,rca,mcp,density,year,pci,eci,coi,cog
6684401,Mozambique,"Horses, asses, mules and hinnies; live",101,0,0.00,0,0.06,2018,0.30,-1.26,-0.65,0.48
6684045,Mozambique,Bovine animals; live,102,0,0.00,0,0.08,2018,0.06,-1.26,-0.65,0.32
6684910,Mozambique,Swine; live,103,0,0.00,0,0.06,2018,1.96,-1.26,-0.65,0.70
6684834,Mozambique,Sheep and goats; live,104,0,0.00,0,0.10,2018,-1.79,-1.26,-0.65,-0.06
6684735,Mozambique,"Poultry; live, fowls of the species gallus domesticus, ducks, geese, turkeys and guinea fowls",105,0,0.00,0,0.07,2018,0.95,-1.26,-0.65,0.53


In [62]:
def cal_distancia(data,
    ):
  "En la siguiente función se realizará el cálculo de COI para cada país del dataframe"
  t_df = data.copy()

  
  #Se realiza el cálculo de propiedades
  print("Se realiza el cálculo de distancia...")

  # t_df['density_r'] = t_df[['Mcp','density']].apply(lambda x: 0 if (x.Mcp==0) else x.density,axis=1 )

  t_df['distancia']=t_df.apply( lambda x: (1-x.density),axis = 1)
  mean =   t_df['distancia'].mean()
  std =   t_df['distancia'].std()

  t_df['dist_norm']=t_df.apply( lambda x: (x.distancia-mean)/std,axis = 1)

  
  return t_df

def normaPCI(data,
    ):
  t_df = data.copy()

  
  #Se realiza el cálculo de propiedades

  mean =   t_df['pci'].mean()
  std =   t_df['pci'].std()
  t_df['pci_norm']=t_df.apply( lambda x: ((x.pci-mean))/std,axis = 1)

  print("Se realiza esuma el right con el COG.")

  return t_df


def normaDensity(data,
    ):
  t_df = data.copy()

  
  #Se realiza el cálculo de propiedades
  print("Se realiza el cálculo de lado derecho...")

  mean =   t_df['density'].mean()
  std =   t_df['density'].std()
  t_df['density_norm']=t_df.apply( lambda x: ((x.density-mean))/std,axis = 1)

  print("Density normalizado! ")

  return t_df

In [76]:
def cal_ogi(data,
    ):
  "En la siguiente función se realizará el cálculo de COI para cada país del dataframe"
  t_df = data.copy()

  
  #Se realiza el cálculo de propiedades
  print("Se realiza el cálculo de lado derecho...")

  # t_df['density_r'] = t_df[['Mcp','density']].apply(lambda x: 0 if (x.Mcp==0) else x.density,axis=1 )

  # t_df['right']=t_df.apply( lambda x: (1-x.dist_norm)*x.pci_norm,axis = 1)
  t_df['right']=t_df.apply( lambda x: (1-x.distancia)*x.pci,axis = 1)
  t_df['right_Nr']=t_df.apply( lambda x: (1-x.dist_norm)*x.pci_norm,axis = 1)



  mean = t_df['right'].mean()
  std = t_df['right'].std()

  meanCog = t_df['cog'].mean()
  stdCog= t_df['cog'].std()

  print("Se realiza la suma el right con el COG.")
  t_df['cog_norm']=t_df.apply( lambda x: (x.cog-meanCog)/stdCog,axis = 1)

  t_df['right_norm']=t_df.apply( lambda x: (x.right-mean)/std,axis = 1)

  t_df['ogi1'] = t_df.apply(lambda x: x.cog + x.right, axis=1)
  meanOgi1 =  t_df['ogi1'].mean()
  stdOgi1 =  t_df['ogi1'].std()
  t_df['ogi'] = t_df.apply(lambda x: (x.ogi1 -meanOgi1)/stdOgi1, axis=1)

  t_df['ogi_norm_inter'] = t_df.apply(lambda x: x.cog_norm + x.right_norm, axis=1)
  meanOgi =  t_df['ogi_norm_inter'].mean()
  stdOgi =  t_df['ogi_norm_inter'].std()
  
  t_df['ogi_norm'] = t_df.apply(lambda x: (x.ogi_norm_inter -meanOgi)/stdOgi, axis=1)

  t_df.drop(['ogi_norm_inter'],axis=1,inplace=True,errors='ignore')


  return t_df

In [79]:
moz2018densityNorm = normaDensity(moz2018)
moz2018pciNorm = normaPCI(moz2018densityNorm)
moz2018Distancia=cal_distancia(moz2018pciNorm)
moz2018Distancia.head()

Se realiza el cálculo de lado derecho...
Density normalizado! 
Se realiza esuma el right con el COG.
Se realiza el cálculo de distancia...


,country_name,product_name,product_code,export_value,rca,mcp,density,year,pci,eci,coi,cog,density_norm,pci_norm,distancia,dist_norm
6684401,Mozambique,"Horses, asses, mules and hinnies; live",101,0,0.00,0,0.06,2018,0.30,-1.26,-0.65,0.48,-0.34,-0.35,0.94,0.34
6684045,Mozambique,Bovine animals; live,102,0,0.00,0,0.08,2018,0.06,-1.26,-0.65,0.32,0.70,-0.50,0.92,-0.70
6684910,Mozambique,Swine; live,103,0,0.00,0,0.06,2018,1.96,-1.26,-0.65,0.70,-0.39,0.64,0.94,0.39
6684834,Mozambique,Sheep and goats; live,104,0,0.00,0,0.10,2018,-1.79,-1.26,-0.65,-0.06,1.37,-1.60,0.90,-1.37
6684735,Mozambique,"Poultry; live, fowls of the species gallus domesticus, ducks, geese, turkeys and guinea fowls",105,0,0.00,0,0.07,2018,0.95,-1.26,-0.65,0.53,0.09,0.04,0.93,-0.09


In [81]:
moz2018Ogi = cal_ogi(moz2018Distancia)
codes=[8524,7326,2402,8468,8546,102,2106,4008, 6806]
moz2018Filter = moz2018Ogi[moz2018Ogi['product_code'].isin(codes)].copy()
moz2018Filter = moz2018Filter[['product_code','product_name','rca','density','density_norm','dist_norm','pci_norm','ogi1','ogi','ogi_norm']].copy()
moz2018Filter.head(7)
moz2018Ogi.describe()

Se realiza el cálculo de lado derecho...
Se realiza la suma el right con el COG.


,product_code,product_name,rca,density,density_norm,dist_norm,pci_norm,ogi1,ogi,ogi_norm
6684045,102,Bovine animals; live,0.00,0.08,0.70,-0.70,-0.50,0.32,-0.47,-0.36
6684314,2106,Food preparations not elsewhere specified or included,0.01,0.08,0.67,-0.67,-0.08,0.39,-0.35,-0.11
6684124,2402,"Cigars, cheroots, cigarillos and cigarettes; of tobacco or of tobacco substitutes",0.01,0.10,1.48,-1.48,-0.73,0.01,-1.10,-0.89
6684722,4008,"Plates, sheets, strip, rods and profile shapes, of vulcanised rubber other than hard rubber",0.01,0.05,-0.70,0.70,1.07,1.24,1.36,1.22
6684856,6806,"Slag, rock wool and similar mineral wools; exfoliated vermiculite, expanded clays, foamed slag, mixtures and articles of heat, sound insulating or sound-absorbing mineral materials",0.03,0.05,-0.41,0.41,0.68,0.98,0.84,0.82
6684449,7326,"Iron or steel; articles, n.e.s. in chapter 73",0.02,0.06,-0.38,0.38,0.97,1.05,0.97,1.00
6684508,8468,"Machinery and apparatus for soldering, brazing, welding, whether or not capable of cutting, other than those of heading no. 8515; gas-operated surface tempering machines and appliances",0.07,0.05,-0.50,0.50,0.74,1.16,1.20,1.07


,product_code,export_value,rca,mcp,density,year,pci,eci,coi,cog,density_norm,pci_norm,distancia,dist_norm,right,right_Nr,cog_norm,right_norm,ogi1,ogi,ogi_norm
count,1220.00,1220.00,1220.00,1220.00,1220.00,1220.00,1220.00,1220.00,1220.00,1220.00,1220.00,1220.00,1220.00,1220.00,1220.00,1220.00,1220.00,1220.00,1220.00,1220.00,1220.00
mean,5297.65,4739084.32,1.45,0.07,0.06,2018.00,0.88,-1.26,-0.65,0.54,-0.00,-0.00,0.94,-0.00,0.02,-0.89,-0.00,0.00,0.56,0.00,-0.00
std,2766.50,53016800.32,12.11,0.26,0.02,0.00,1.67,0.00,0.00,0.40,1.00,1.00,0.02,1.00,0.12,2.05,1.00,1.00,0.50,1.00,1.00
min,101.00,0.00,0.00,0.00,0.03,2018.00,-5.31,-1.26,-0.65,-0.36,-1.59,-3.71,0.82,-4.73,-0.85,-18.55,-2.25,-7.20,-1.21,-3.52,-4.98
25%,2843.75,0.00,0.00,0.00,0.05,2018.00,-0.25,-1.26,-0.65,0.20,-0.74,-0.68,0.92,-0.53,-0.02,-0.97,-0.83,-0.33,0.18,-0.75,-0.62
50%,5356.00,1500.00,0.00,0.00,0.06,2018.00,1.01,-1.26,-0.65,0.55,-0.28,0.08,0.94,0.28,0.06,-0.08,0.04,0.31,0.61,0.09,0.19
75%,8104.25,63832.75,0.03,0.00,0.08,2018.00,2.12,-1.26,-0.65,0.85,0.53,0.74,0.95,0.74,0.10,0.15,0.77,0.68,0.96,0.79,0.77
max,9999.00,1437063424.00,276.86,1.00,0.18,2018.00,5.92,-1.26,-0.65,1.41,4.73,3.02,0.97,1.59,0.16,0.66,2.17,1.13,1.54,1.95,1.66


In [21]:
moz2018Filter.head(7)


,product_code,product_name,rca,density,density_norm,pci_norm,ogi_norm


##Librerías de Hidalgo

In [22]:
!pip install economic-complexity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
import economic_complexity as ecplx


In [24]:
df_base_2018 = df_base[df_base['year']==2018].copy()

In [25]:
df_pivot = pd.pivot_table(df_base_2018, index=['country_name'],
                                     columns=['product_code'],
                                     values='export_value')\
             .reset_index()\
             .set_index('country_name')\
             .dropna(axis=1, how="all")\
             .fillna(0)\
             .astype(float)

In [26]:
df_pivot.head()

product_code,0101,0102,0103,0104,0105,0106,0201,0202,0203,0204,0205,0206,0207,0208,0209,0210,0301,0302,0303,0304,0305,0306,0307,0401,0402,0403,0404,0405,0406,0407,0408,0409,0410,0501,0502,0504,0505,0506,0507,0508,0510,0511,0601,0602,0603,0604,0701,0702,0703,0704,0705,0706,0707,0708,0709,0710,0711,0712,0713,0714,0801,0802,0803,0804,0805,0806,0807,0808,0809,0810,0811,0812,0813,0814,0901,0902,0903,0904,0905,0906,0907,0908,0909,0910,1001,1002,1003,1004,1005,1006,1007,1008,1101,1102,1103,1104,1105,1106,1107,1108,1109,1201,1202,1203,1204,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214,1301,1302,1401,1404,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,1520,1521,1522,1601,1602,1603,1604,1605,1701,1702,1703,1704,1801,1802,1803,1804,1805,1806,1901,1902,1903,1904,1905,2001,2002,2003,2004,2005,2006,2007,2008,2009,2101,2102,2103,2104,2105,2106,2201,2202,2203,2204,2205,2206,2207,2208,2209,2301,2302,2303,2304,2305,2306,2307,2308,2309,2401,2402,2403,2501,2502,2503,2504,2505,2506,2507,2508,2509,2510,2511,2512,2513,2514,2515,2516,2517,2518,2519,2520,2521,2522,2523,2524,2525,2526,2528,2529,2530,2601,2602,2603,2604,2605,2606,2607,2608,2609,2610,2611,2612,2613,2614,2615,2616,2617,2618,2619,2620,2621,2701,2702,2703,...,8421,8422,8423,8424,8425,8426,8427,8428,8429,8430,8431,8432,8433,8434,8435,8436,8437,8438,8439,8440,8441,8442,8443,8444,8445,8446,8447,8448,8449,8450,8451,8452,8453,8454,8455,8456,8457,8458,8459,8460,8461,8462,8463,8464,8465,8466,8467,8468,8469,8470,8471,8472,8473,8474,8475,8476,8477,8478,8479,8480,8481,8482,8483,8484,8485,8501,8502,8503,8504,8505,8506,8507,8508,8509,8510,8511,8512,8513,8514,8515,8516,8517,8518,8519,8520,8521,8522,8523,8524,8525,8526,8527,8528,8529,8530,8531,8532,8533,8534,8535,8536,8537,8538,8539,8540,8541,8542,8543,8544,8545,8546,8547,8548,8601,8602,8603,8604,8605,8606,8607,8608,8609,8701,8702,8703,8704,8705,8706,8707,8708,8709,8710,8711,8712,8713,8714,8715,8716,8801,8802,8803,8804,8805,8901,8902,8903,8904,8905,8906,8907,8908,9001,9002,9003,9004,9005,9006,9007,9008,9009,9010,9011,9012,9013,9014,9015,9016,9017,9018,9019,9020,9021,9022,9023,9024,9025,9026,9027,9028,9029,9030,9031,9032,9033,9101,9102,9103,9104,9105,9106,9107,9108,9109,9110,9111,9112,9113,9114,9201,9202,9205,9206,9207,9208,9209,9301,9302,9303,9304,9305,9306,9307,9401,9402,9403,9404,9405,9406,9502,9503,9504,9505,9506,9507,9508,9601,9602,9603,9604,9605,9606,9607,9608,9609,9610,9611,9612,9613,9614,9615,9616,9617,9618,9701,9702,9703,9704,9705,9706,9999
country_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.00,130333.00,0.00,2659886.00,0.00,713231.00,0.00,0.00,0.00,92062.00,0.00,3413.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4139.00,0.00,5569.00,38858.00,5359.00,0.00,0.00,0.00,0.00,10427.00,19082.00,720499.00,0.00,4043205.00,0.00,173128.00,0.00,0.00,0.00,16112.00,0.00,12095.00,481874.00,0.00,254972.00,28416918.00,9461113.00,215111.00,9676.00,7097.00,10843511.00,15304.00,434493.00,149812.00,0.00,314690.00,67298056.00,23522.00,43104.00,67236816.00,2621.00,62823712.00,459527.00,200201264.00,1679515.00,21835542.00,16994596.00,27987412.00,439629.00,884391.00,28342332.00,0.00,0.00,72790.00,867394.00,6794.00,0.00,0.00,0.00,0.00,15419482.00,20157944.00,3184.00,0.00,464393.00,0.00,0.00,0.00,300769.00,83926.00,0.00,6321.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2161.00,459245.00,16853.00,504204.00,0.00,13807.00,13842392.00,0.00,7262882.00,0.00,9722914.00,1980272.00,0.00,1624271.00,78261392.00,5112418.00,195094.00,227643.00,0.00,0.00,0.00,0.00,0.00

### RCA, ECI, PCI, Proximity, Relatedness, Distance

In [27]:
rca = ecplx.rca(df_pivot)
ECI, PCI = ecplx.complexity(rca, iterations=20, drop=True)
proximityValues = ecplx.proximity(rca)
relatedness = ecplx.relatedness(rca,proximityValues)
distance= ecplx.distance(rca, proximityValues)
distance.head()

product_code,0101,0102,0103,0104,0105,0106,0201,0202,0203,0204,0205,0206,0207,0208,0209,0210,0301,0302,0303,0304,0305,0306,0307,0401,0402,0403,0404,0405,0406,0407,0408,0409,0410,0501,0502,0504,0505,0506,0507,0508,0510,0511,0601,0602,0603,0604,0701,0702,0703,0704,0705,0706,0707,0708,0709,0710,0711,0712,0713,0714,0801,0802,0803,0804,0805,0806,0807,0808,0809,0810,0811,0812,0813,0814,0901,0902,0903,0904,0905,0906,0907,0908,0909,0910,1001,1002,1003,1004,1005,1006,1007,1008,1101,1102,1103,1104,1105,1106,1107,1108,1109,1201,1202,1203,1204,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214,1301,1302,1401,1404,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,1520,1521,1522,1601,1602,1603,1604,1605,1701,1702,1703,1704,1801,1802,1803,1804,1805,1806,1901,1902,1903,1904,1905,2001,2002,2003,2004,2005,2006,2007,2008,2009,2101,2102,2103,2104,2105,2106,2201,2202,2203,2204,2205,2206,2207,2208,2209,2301,2302,2303,2304,2305,2306,2307,2308,2309,2401,2402,2403,2501,2502,2503,2504,2505,2506,2507,2508,2509,2510,2511,2512,2513,2514,2515,2516,2517,2518,2519,2520,2521,2522,2523,2524,2525,2526,2528,2529,2530,2601,2602,2603,2604,2605,2606,2607,2608,2609,2610,2611,2612,2613,2614,2615,2616,2617,2618,2619,2620,2621,2701,2702,2703,...,8421,8422,8423,8424,8425,8426,8427,8428,8429,8430,8431,8432,8433,8434,8435,8436,8437,8438,8439,8440,8441,8442,8443,8444,8445,8446,8447,8448,8449,8450,8451,8452,8453,8454,8455,8456,8457,8458,8459,8460,8461,8462,8463,8464,8465,8466,8467,8468,8469,8470,8471,8472,8473,8474,8475,8476,8477,8478,8479,8480,8481,8482,8483,8484,8485,8501,8502,8503,8504,8505,8506,8507,8508,8509,8510,8511,8512,8513,8514,8515,8516,8517,8518,8519,8520,8521,8522,8523,8524,8525,8526,8527,8528,8529,8530,8531,8532,8533,8534,8535,8536,8537,8538,8539,8540,8541,8542,8543,8544,8545,8546,8547,8548,8601,8602,8603,8604,8605,8606,8607,8608,8609,8701,8702,8703,8704,8705,8706,8707,8708,8709,8710,8711,8712,8713,8714,8715,8716,8801,8802,8803,8804,8805,8901,8902,8903,8904,8905,8906,8907,8908,9001,9002,9003,9004,9005,9006,9007,9008,9009,9010,9011,9012,9013,9014,9015,9016,9017,9018,9019,9020,9021,9022,9023,9024,9025,9026,9027,9028,9029,9030,9031,9032,9033,9101,9102,9103,9104,9105,9106,9107,9108,9109,9110,9111,9112,9113,9114,9201,9202,9205,9206,9207,9208,9209,9301,9302,9303,9304,9305,9306,9307,9401,9402,9403,9404,9405,9406,9502,9503,9504,9505,9506,9507,9508,9601,9602,9603,9604,9605,9606,9607,9608,9609,9610,9611,9612,9613,9614,9615,9616,9617,9618,9701,9702,9703,9704,9705,9706,9999
country_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.94,0.93,0.95,0.89,0.94,0.89,0.93,0.92,0.96,0.90,0.93,0.93,0.94,0.94,0.95,0.95,0.91,0.91,0.90,0.92,0.92,0.90,0.90,0.94,0.92,0.93,0.95,0.94,0.93,0.92,0.95,0.91,0.94,0.91,0.96,0.90,0.96,0.92,0.92,0.92,0.90,0.91,0.93,0.93,0.93,0.92,0.92,0.90,0.87,0.90,0.92,0.91,0.91,0.89,0.90,0.92,0.91,0.92,0.87,0.90,0.90,0.89,0.90,0.89,0.90,0.90,0.89,0.91,0.89,0.89,0.92,0.91,0.90,0.91,0.91,0.89,0.84,0.90,0.93,0.92,0.92,0.91,0.89,0.89,0.93,0.96,0.94,0.94,0.93,0.90,0.89,0.91,0.90,0.91,0.92,0.92,0.95,0.89,0.94,0.93,0.96,0.91,0.87,0.91,0.91,0.95,0.94,0.87,0.92,0.91,0.96,0.88,0.89,0.94,0.91,0.87,0.91,0.91,0.89,0.96,0.93,0.95,0.91,0.95,0.95,0.92,0.91,0.92,0.93,0.92,0.91,0.93,0.95,0.91,0.91,0.92,0.93,0.94,0.94,0.93,0.94,0.94,0.93,0.95,0.92,0.91,0.90,0.94,0.90,0.92,0.91,0.93,0.96,0.95,0.94,0.93,0.94,0.91,0.92,0.92,0.92,0.91,0.91,0.96,0.94,0.91,0.93,0.92,0.90,0.90,0.92,0.92,0.92,0.93,0.93,0.93,0.93,0.92,0.93,0.92,0.94,0.94,0.92,0.92,0.93,0.91,0.90,0.94,0.93,0.91,0.92,0.

In [28]:
cogHidal = ecplx.opportunity_gain(rcas=rca,
                                proximities=proximityValues,
                                pci=PCI)

In [29]:
CogHidalMozam = cogHidal.loc[('Mozambique'),:]
cogHidalMozam = CogHidalMozam.to_frame(name="cog").reset_index()
cogHidalMozam.describe()

,cog
count,1220.00
mean,0.18
std,0.14
min,-0.16
25%,0.07
50%,0.18
75%,0.28
max,0.72


#### OGI Y COG MANUAL

In [30]:
rcas = rca.copy()
rcas[rcas >= 1] = 1
rcas[rcas <1] = 0
inverse_rcas = 1 - rcas
prox_ratio = proximityValues / proximityValues.sum()
middle = inverse_rcas.multiply(PCI)
opp_gain=middle.dot(prox_ratio)
Ogi = opp_gain
OgiMozam = Ogi.loc[('Mozambique'),:]

ogi = OgiMozam.to_frame(name="ogi").reset_index()
# ogi.head()
ogi.describe()

,ogi
count,1220.00
mean,0.16
std,0.20
min,-0.50
25%,0.02
50%,0.18
75%,0.31
max,0.80


In [31]:
dcp = ecplx.relatedness(inverse_rcas,proximityValues )
    # now get the inverse
dcp = 1 - dcp
    # we now have the right-half of the equation
right = dcp.multiply(PCI)
# right.head()
cog = opp_gain + right
cog.head()

product_code,0101,0102,0103,0104,0105,0106,0201,0202,0203,0204,0205,0206,0207,0208,0209,0210,0301,0302,0303,0304,0305,0306,0307,0401,0402,0403,0404,0405,0406,0407,0408,0409,0410,0501,0502,0504,0505,0506,0507,0508,0510,0511,0601,0602,0603,0604,0701,0702,0703,0704,0705,0706,0707,0708,0709,0710,0711,0712,0713,0714,0801,0802,0803,0804,0805,0806,0807,0808,0809,0810,0811,0812,0813,0814,0901,0902,0903,0904,0905,0906,0907,0908,0909,0910,1001,1002,1003,1004,1005,1006,1007,1008,1101,1102,1103,1104,1105,1106,1107,1108,1109,1201,1202,1203,1204,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214,1301,1302,1401,1404,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,1520,1521,1522,1601,1602,1603,1604,1605,1701,1702,1703,1704,1801,1802,1803,1804,1805,1806,1901,1902,1903,1904,1905,2001,2002,2003,2004,2005,2006,2007,2008,2009,2101,2102,2103,2104,2105,2106,2201,2202,2203,2204,2205,2206,2207,2208,2209,2301,2302,2303,2304,2305,2306,2307,2308,2309,2401,2402,2403,2501,2502,2503,2504,2505,2506,2507,2508,2509,2510,2511,2512,2513,2514,2515,2516,2517,2518,2519,2520,2521,2522,2523,2524,2525,2526,2528,2529,2530,2601,2602,2603,2604,2605,2606,2607,2608,2609,2610,2611,2612,2613,2614,2615,2616,2617,2618,2619,2620,2621,2701,2702,2703,...,8421,8422,8423,8424,8425,8426,8427,8428,8429,8430,8431,8432,8433,8434,8435,8436,8437,8438,8439,8440,8441,8442,8443,8444,8445,8446,8447,8448,8449,8450,8451,8452,8453,8454,8455,8456,8457,8458,8459,8460,8461,8462,8463,8464,8465,8466,8467,8468,8469,8470,8471,8472,8473,8474,8475,8476,8477,8478,8479,8480,8481,8482,8483,8484,8485,8501,8502,8503,8504,8505,8506,8507,8508,8509,8510,8511,8512,8513,8514,8515,8516,8517,8518,8519,8520,8521,8522,8523,8524,8525,8526,8527,8528,8529,8530,8531,8532,8533,8534,8535,8536,8537,8538,8539,8540,8541,8542,8543,8544,8545,8546,8547,8548,8601,8602,8603,8604,8605,8606,8607,8608,8609,8701,8702,8703,8704,8705,8706,8707,8708,8709,8710,8711,8712,8713,8714,8715,8716,8801,8802,8803,8804,8805,8901,8902,8903,8904,8905,8906,8907,8908,9001,9002,9003,9004,9005,9006,9007,9008,9009,9010,9011,9012,9013,9014,9015,9016,9017,9018,9019,9020,9021,9022,9023,9024,9025,9026,9027,9028,9029,9030,9031,9032,9033,9101,9102,9103,9104,9105,9106,9107,9108,9109,9110,9111,9112,9113,9114,9201,9202,9205,9206,9207,9208,9209,9301,9302,9303,9304,9305,9306,9307,9401,9402,9403,9404,9405,9406,9502,9503,9504,9505,9506,9507,9508,9601,9602,9603,9604,9605,9606,9607,9608,9609,9610,9611,9612,9613,9614,9615,9616,9617,9618,9701,9702,9703,9704,9705,9706,9999
country_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.15,-0.02,0.24,-0.32,0.08,-0.35,-0.01,-0.15,0.35,-0.35,0.01,-0.01,0.10,0.07,0.24,0.23,-0.25,-0.35,-0.47,-0.32,-0.35,-0.57,-0.42,0.06,-0.14,0.01,0.25,0.13,0.06,-0.08,0.21,-0.25,0.06,-0.02,0.21,-0.16,0.30,0.04,-0.17,-0.16,-0.39,-0.18,0.03,-0.07,-0.25,-0.15,-0.04,-0.27,-0.45,-0.13,-0.03,-0.14,-0.16,-0.49,-0.35,-0.14,-0.21,-0.12,-0.56,-0.45,-0.56,-0.40,-0.50,-0.55,-0.30,-0.17,-0.47,-0.04,-0.30,-0.33,-0.08,-0.21,-0.17,-0.29,-0.31,-0.53,-0.62,-0.43,-0.21,-0.22,-0.34,-0.43,-0.26,-0.42,0.01,0.30,0.13,0.15,-0.08,-0.51,-0.43,-0.18,-0.37,-0.26,-0.17,-0.12,0.32,-0.43,0.13,0.03,0.39,-0.16,-0.62,-0.28,-0.09,0.16,0.05,-0.57,-0.12,-0.16,0.33,-0.53,-0.45,0.11,-0.02,-0.71,-0.19,-0.19,-0.44,0.36,0.03,0.27,-0.47,0.23,0.27,-0.18,-0.28,-0.07,0.03,-0.50,-0.18,-0.36,0.24,-0.15,-0.33,-0.23,-0.02,0.12,0.09,-0.18,0.05,0.10,0.02,0.24,-0.35,-0.41,-0.44,0.09,-0.34,-0.11,-0.56,-0.44,0.23,0.09,0.02,0.04,0.05,-0.25,-0.21,-0.07,-0.06,-0.19,-0.22,0.27,0.19,-0.17,-0.04,-0.08,-0

In [32]:
CogMozam = cog.loc[('Mozambique'),:]
cogMozam = CogMozam.to_frame(name="cog").reset_index()
moz2018=moz2018.sort_values(by=['product_code'])
eci = ECI.to_frame(name="ECI").reset_index()
pci = PCI.to_frame(name="PCI").reset_index()

eciMozambique=eci[eci['country_name']=='Mozambique']
cogMozam.head()
moz2018.head()
cogHidalMozam.head()
eciMozambique.head()
pci.describe()

,product_code,cog
0,0101,0.15
1,0102,-0.01
2,0103,0.26
3,0104,-0.31
4,0105,0.09


,country_name,product_name,product_code,export_value,rca,mcp,density,year,pci,eci,coi,cog
6684401,Mozambique,"Horses, asses, mules and hinnies; live",0101,0,0.00,0,0.06,2018,0.30,-1.26,-0.65,0.48
6684045,Mozambique,Bovine animals; live,0102,0,0.00,0,0.08,2018,0.06,-1.26,-0.65,0.32
6684910,Mozambique,Swine; live,0103,0,0.00,0,0.06,2018,1.96,-1.26,-0.65,0.70
6684834,Mozambique,Sheep and goats; live,0104,0,0.00,0,0.10,2018,-1.79,-1.26,-0.65,-0.06
6684735,Mozambique,"Poultry; live, fowls of the species gallus domesticus, ducks, geese, turkeys and guinea fowls",0105,0,0.00,0,0.07,2018,0.95,-1.26,-0.65,0.53


,product_code,cog
0,0101,0.19
1,0102,0.07
2,0103,0.19
3,0104,0.01
4,0105,0.09


,country_name,ECI
137,Mozambique,-1.26


,PCI
count,1220.00
mean,-0.00
std,1.00
min,-3.73
25%,-0.68
50%,0.08
75%,0.74
max,3.01


### Se descarga el dataframe de ECI y PCI a la máquina: 


In [33]:
# from google.colab import files
# df_ecm2.to_csv('df_ecm3.csv') 
# files.download('df_ecm3.csv')